In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

if os.getenv("HATCHET_CLIENT_TOKEN", None) is None:
    print(
        "HATCHET_CLIENT_TOKEN is required. Please set it in your environment variables."
    )
    exit(1)


## Chat OTel

In [2]:
from hatchet_schemas import ChatOtelInput, ChatOtelOutput

question = "Where does Anna Karenina take place?"
input = ChatOtelInput(question=question)

### Local

In [ ]:
from hatchet_playground.workflows.chat_otel import langfuse_task

result: ChatOtelOutput = await langfuse_task.aio_run(input=input)
print(result.answer)


"Anna Karenina," the novel by Leo Tolstoy, primarily takes place in Russia during the late 19th century. The story moves through various settings, including Moscow and St. Petersburg, which are central to the social life of the characters. The novel captures the contrast between urban and rural life, and locations such as the countryside estates and railway stations also play significant roles in the narrative. The rich descriptions of these settings highlight the cultural and social backdrop of Russian society at that time.


In [ ]:
from langfuse import get_client
from opentelemetry.trace import StatusCode

tracer = get_client()


async def custom_span() -> None:
    # Traces will send to Langfuse
    # Use `_otel_tracer` to access the OpenTelemetry tracer if you need
    # to e.g. log statuses or attributes manually.

    with tracer._otel_tracer.start_as_current_span(name="general_nb_local") as span:
        result = await langfuse_task.aio_run(input=input)
        answer = result.answer

        if not answer:
            span.set_status(StatusCode.ERROR)
            return

        span.set_attribute("answer", answer)


await custom_span()

### External

In [ ]:
from hatchet_playground.external.runner import ExternalTaskRunner
from opentelemetry.trace import get_tracer_provider
from hatchet_sdk import Hatchet
from hatchet_sdk.opentelemetry.instrumentor import HatchetInstrumentor
from langfuse import Langfuse
from opentelemetry.trace import get_tracer_provider

## TODO: This may not be needed in the client side? Research it !!
# lf = Langfuse()

HatchetInstrumentor(tracer_provider=get_tracer_provider()).instrument()

# Initialize Hatchet client
hatchet = Hatchet(debug=False)


In [ ]:
# Without Opentelemetry

question = "Where does Anna Karenina take place?"
runner = ExternalTaskRunner(
    task_name="chat-otel",
    input_payload={"question": question},
    hatchet=None,
    # hatchet=hatchet,
)

workflow_run_ref = await runner.trigger_no_wait()
_ = await runner.wait_for_terminal_status(workflow_run_ref.workflow_run_id)
result = await workflow_run_ref.aio_result()
print(result.answer)


workflow_run_id=9a394a39-4653-436e-9695-4977c11c2020 status=QUEUED
workflow_run_id=9a394a39-4653-436e-9695-4977c11c2020 status=RUNNING
workflow_run_id=9a394a39-4653-436e-9695-4977c11c2020 status=RUNNING
workflow_run_id=9a394a39-4653-436e-9695-4977c11c2020 status=RUNNING
workflow_run_id=9a394a39-4653-436e-9695-4977c11c2020 status=RUNNING
workflow_run_id=9a394a39-4653-436e-9695-4977c11c2020 status=COMPLETED
"Anna Karenina," the novel by Leo Tolstoy, primarily takes place in late 19th-century Russia, particularly in Moscow and St. Petersburg. The story explores the lives of its central characters as they navigate the complexities of Russian society, relationships, and personal dilemmas. The settings are crucial to understanding the cultural and social dynamics that influence the characters’ actions and fates throughout the narrative.


In [ ]:
# With Opentelemetry

from langfuse import get_client
from opentelemetry.trace import StatusCode

tracer = get_client()


async def custom_span() -> None:
    # Traces will send to Langfuse
    # Use `_otel_tracer` to access the OpenTelemetry tracer if you need
    # to e.g. log statuses or attributes manually.

    with tracer._otel_tracer.start_as_current_span(name="general_nb_external") as span:
        question = "Where does Anna Karenina take place?"
        runner = ExternalTaskRunner(
            task_name="chat-otel",
            input_payload={"question": question},
            # hatchet=None,
            hatchet=hatchet,
        )

        workflow_run_ref = await runner.trigger_no_wait()
        _ = await runner.wait_for_terminal_status(workflow_run_ref.workflow_run_id)
        result = await workflow_run_ref.aio_result()
        print(result.answer)

        answer = result.answer

        if not answer:
            span.set_status(StatusCode.ERROR)
            return

        span.set_attribute("answer", answer)


await custom_span()